In [35]:
#Progetto FOCS

You have to work on the [University dataset](https://drive.google.com/drive/folders/1Hs3nRtK_F3h8eg59B4-TD1DEua6g8Klv). It contains three different university rankings:
- The Times Higher Education World University Ranking, shortly *Times*,
- the Academic Ranking of World Universities, shortly *Shanghai*,
- the Center for World University Rankings, shortly *cwur*.

Notes
1. It is mandatory to use GitHub for developing the project.
2. The project must be a jupyter notebook.
3. There is no restriction on the libraries that can be used, nor on the Python version.
4. All questions on the project **must** be asked in a public channel on [Zulip](https://focs.zulipchat.com).

In [36]:
import pandas as pd
import re
import numpy as np

In [37]:
times = pd.read_csv('timesData.csv')
times.head()

,world_rank,university_name,country,teaching,international,research,citations,income,total_score,num_students,student_staff_ratio,international_students,female_male_ratio,year
0,1,Harvard University,United States of America,99.7,72.4,98.7,98.8,34.5,96.1,"20,152",8.9,25%,NaN,2011
1,2,California Institute of Technology,United States of America,97.7,54.6,98.0,99.9,83.7,96.0,"2,243",6.9,27%,33 : 67,2011
2,3,Massachusetts Institute of Technology,United States of America,97.8,82.3,91.4,99.9,87.5,95.6,"11,074",9.0,33%,37 : 63,2011
3,4,Stanford University,United States of America,98.3,29.5,98.1,99.2,64.3,94.3,"15,596",7.8,22%,42 : 58,2011
4,5,Princeton University,United States of America,90.9,70.3,95.4,99.9,-,94.2,"7,929",8.4,27%,45 : 55,2011


In [38]:
shanghai = pd.read_csv('shanghaiData.csv')
shanghai.head()

,world_rank,university_name,national_rank,total_score,alumni,award,hici,ns,pub,pcp,year
0,1,Harvard University,1,100.0,100.0,100.0,100.0,100.0,100.0,72.4,2005
1,2,University of Cambridge,1,73.6,99.8,93.4,53.3,56.6,70.9,66.9,2005
2,3,Stanford University,2,73.4,41.1,72.2,88.5,70.9,72.3,65.0,2005
3,4,"University of California, Berkeley",3,72.8,71.8,76.0,69.4,73.9,72.2,52.7,2005
4,5,Massachusetts Institute of Technology (MIT),4,70.1,74.0,80.6,66.7,65.8,64.3,53.0,2005


In [39]:
cwur = pd.read_csv('cwurData.csv')
cwur.head()

,world_rank,institution,country,national_rank,quality_of_education,alumni_employment,quality_of_faculty,publications,influence,citations,broad_impact,patents,score,year
0,1,Harvard University,USA,1,7,9,1,1,1,1,NaN,5,100.00,2012
1,2,Massachusetts Institute of Technology,USA,2,9,17,3,12,4,4,NaN,1,91.67,2012
2,3,Stanford University,USA,3,17,11,5,4,2,2,NaN,15,89.50,2012
3,4,University of Cambridge,United Kingdom,1,10,24,4,16,16,11,NaN,50,86.17,2012
4,5,California Institute of Technology,USA,4,2,29,7,37,22,22,NaN,18,85.21,2012


Check of Na values 

In [40]:
times.isnull().any()

world_rank                False
university_name           False
country                   False
teaching                  False
international             False
research                  False
citations                 False
income                    False
total_score               False
num_students               True
student_staff_ratio        True
international_students     True
female_male_ratio          True
year                      False
dtype: bool

In [41]:
shanghai.isnull().any()

world_rank         False
university_name     True
national_rank       True
total_score         True
alumni              True
award               True
hici                True
ns                  True
pub                 True
pcp                 True
year               False
dtype: bool

In [42]:
cwur.isnull().any()

world_rank              False
institution             False
country                 False
national_rank           False
quality_of_education    False
alumni_employment       False
quality_of_faculty      False
publications            False
influence               False
citations               False
broad_impact             True
patents                 False
score                   False
year                    False
dtype: bool

## 1. For each university, extract from the times dataset the most recent and the least recent data, obtaining two separate dataframes

In [43]:
times['year'].dtype

dtype('int64')

In [44]:
times[times['university_name'] == 'Harvard University']

,world_rank,university_name,country,teaching,international,research,citations,income,total_score,num_students,student_staff_ratio,international_students,female_male_ratio,year
0,1,Harvard University,United States of America,99.7,72.4,98.7,98.8,34.5,96.1,"20,152",8.9,25%,NaN,2011
201,2,Harvard University,United States of America,95.8,67.5,97.4,99.8,35.9,93.9,"20,152",8.9,25%,NaN,2012
605,4,Harvard University,United States of America,94.9,63.7,98.6,99.2,39.9,93.6,"20,152",8.9,25%,NaN,2013
1003,2,Harvard University,United States of America,95.3,66.2,98.5,99.1,40.6,93.9,"20,152",8.9,25%,NaN,2014
1403,2,Harvard University,United States of America,92.9,67.6,98.6,98.9,44.0,93.3,"20,152",8.9,25%,NaN,2015
1808,6,Harvard University,United States of America,83.6,77.2,99.0,99.8,45.2,91.6,"20,152",8.9,25%,NaN,2016


In [45]:
times_max_year = times.iloc[times.groupby('university_name').idxmax()['year']]
times_max_year.head()

,world_rank,university_name,country,teaching,international,research,citations,income,total_score,num_students,student_staff_ratio,international_students,female_male_ratio,year
2405,601-800,AGH University of Science and Technology,Poland,14.2,17.9,3.7,35.7,-,-,"35,569",17.0,1%,-,2016
2003,201-250,Aalborg University,Denmark,25.1,71.0,28.4,73.8,43.7,-,"17,422",15.9,15%,48 : 52,2016
2056,251-300,Aalto University,Finland,31.1,65.4,32.8,62.1,61.6,-,"16,099",24.2,17%,32 : 68,2016
1908,=106,Aarhus University,Denmark,36.9,76.8,50.7,79.8,68.3,57.7,"23,895",13.6,14%,54 : 46,2016
2105,301-350,Aberystwyth University,United Kingdom,21.6,72.2,18.9,67.2,31.3,-,"9,252",19.2,18%,48 : 52,2016


In [46]:
times_min_year = times.iloc[times.groupby('university_name').idxmin()['year']]
times_min_year.head()

,world_rank,university_name,country,teaching,international,research,citations,income,total_score,num_students,student_staff_ratio,international_students,female_male_ratio,year
2405,601-800,AGH University of Science and Technology,Poland,14.2,17.9,3.7,35.7,-,-,"35,569",17.0,1%,-,2016
501,301-350,Aalborg University,Denmark,19.0,75.3,20.0,27.1,36.4,-,"17,422",15.9,15%,48 : 52,2012
502,301-350,Aalto University,Finland,26.2,49.0,22.2,37.5,61.9,-,"16,099",24.2,17%,32 : 68,2012
166,167,Aarhus University,Denmark,38.1,33.4,55.6,57.3,61.5,49.9,"23,895",13.6,14%,54 : 46,2011
476,276-300,Aberystwyth University,United Kingdom,19.8,63.8,15.5,56.6,35.5,-,"9,252",19.2,18%,48 : 52,2012


## 2. For each university, compute the improvement in income between the least recent and the most recent data points

In [47]:
merged = pd.merge(times_min_year, times_max_year, on = ['university_name', 'country'], suffixes = ['_min', '_max'])
merged.head()

,world_rank_min,university_name,country,teaching_min,international_min,research_min,citations_min,income_min,total_score_min,num_students_min,...,international_max,research_max,citations_max,income_max,total_score_max,num_students_max,student_staff_ratio_max,international_students_max,female_male_ratio_max,year_max
0,601-800,AGH University of Science and Technology,Poland,14.2,17.9,3.7,35.7,-,-,"35,569",...,17.9,3.7,35.7,-,-,"35,569",17.0,1%,-,2016
1,301-350,Aalborg University,Denmark,19.0,75.3,20.0,27.1,36.4,-,"17,422",...,71.0,28.4,73.8,43.7,-,"17,422",15.9,15%,48 : 52,2016
2,301-350,Aalto University,Finland,26.2,49.0,22.2,37.5,61.9,-,"16,099",...,65.4,32.8,62.1,61.6,-,"16,099",24.2,17%,32 : 68,2016
3,167,Aarhus University,Denmark,38.1,33.4,55.6,57.3,61.5,49.9,"23,895",...,76.8,50.7,79.8,68.3,57.7,"23,895",13.6,14%,54 : 46,2016
4,276-300,Aberystwyth University,United Kingdom,19.8,63.8,15.5,56.6,35.5,-,"9,252",...,72.2,18.9,67.2,31.3,-,"9,252",19.2,18%,48 : 52,2016


In [48]:
merged[merged['university_name'] == 'Harvard University']['income_max'].dtype

dtype('O')

In [49]:
def difference(row):
    if (row['income_max'] == '-') or (row['income_min'] == '-'):
        return 'data not available'
    else:
        return float(row['income_max']) - float(row['income_min'])

In [50]:
merged['difference_for'] = merged.apply(difference, axis = 1)
merged.head()

,world_rank_min,university_name,country,teaching_min,international_min,research_min,citations_min,income_min,total_score_min,num_students_min,...,research_max,citations_max,income_max,total_score_max,num_students_max,student_staff_ratio_max,international_students_max,female_male_ratio_max,year_max,difference_for
0,601-800,AGH University of Science and Technology,Poland,14.2,17.9,3.7,35.7,-,-,"35,569",...,3.7,35.7,-,-,"35,569",17.0,1%,-,2016,data not available
1,301-350,Aalborg University,Denmark,19.0,75.3,20.0,27.1,36.4,-,"17,422",...,28.4,73.8,43.7,-,"17,422",15.9,15%,48 : 52,2016,7.3
2,301-350,Aalto University,Finland,26.2,49.0,22.2,37.5,61.9,-,"16,099",...,32.8,62.1,61.6,-,"16,099",24.2,17%,32 : 68,2016,-0.3
3,167,Aarhus University,Denmark,38.1,33.4,55.6,57.3,61.5,49.9,"23,895",...,50.7,79.8,68.3,57.7,"23,895",13.6,14%,54 : 46,2016,6.8
4,276-300,Aberystwyth University,United Kingdom,19.8,63.8,15.5,56.6,35.5,-,"9,252",...,18.9,67.2,31.3,-,"9,252",19.2,18%,48 : 52,2016,-4.2


In [51]:
merged['income_max'] = pd.to_numeric(merged['income_max'], errors = 'coerce')
merged['income_min'] = pd.to_numeric(merged['income_min'], errors = 'coerce')
merged.head()

,world_rank_min,university_name,country,teaching_min,international_min,research_min,citations_min,income_min,total_score_min,num_students_min,...,research_max,citations_max,income_max,total_score_max,num_students_max,student_staff_ratio_max,international_students_max,female_male_ratio_max,year_max,difference_for
0,601-800,AGH University of Science and Technology,Poland,14.2,17.9,3.7,35.7,NaN,-,"35,569",...,3.7,35.7,NaN,-,"35,569",17.0,1%,-,2016,data not available
1,301-350,Aalborg University,Denmark,19.0,75.3,20.0,27.1,36.4,-,"17,422",...,28.4,73.8,43.7,-,"17,422",15.9,15%,48 : 52,2016,7.3
2,301-350,Aalto University,Finland,26.2,49.0,22.2,37.5,61.9,-,"16,099",...,32.8,62.1,61.6,-,"16,099",24.2,17%,32 : 68,2016,-0.3
3,167,Aarhus University,Denmark,38.1,33.4,55.6,57.3,61.5,49.9,"23,895",...,50.7,79.8,68.3,57.7,"23,895",13.6,14%,54 : 46,2016,6.8
4,276-300,Aberystwyth University,United Kingdom,19.8,63.8,15.5,56.6,35.5,-,"9,252",...,18.9,67.2,31.3,-,"9,252",19.2,18%,48 : 52,2016,-4.2


In [52]:
merged['difference'] = merged['income_max'] - merged['income_min']
merged.head()

,world_rank_min,university_name,country,teaching_min,international_min,research_min,citations_min,income_min,total_score_min,num_students_min,...,citations_max,income_max,total_score_max,num_students_max,student_staff_ratio_max,international_students_max,female_male_ratio_max,year_max,difference_for,difference
0,601-800,AGH University of Science and Technology,Poland,14.2,17.9,3.7,35.7,NaN,-,"35,569",...,35.7,NaN,-,"35,569",17.0,1%,-,2016,data not available,NaN
1,301-350,Aalborg University,Denmark,19.0,75.3,20.0,27.1,36.4,-,"17,422",...,73.8,43.7,-,"17,422",15.9,15%,48 : 52,2016,7.3,7.3
2,301-350,Aalto University,Finland,26.2,49.0,22.2,37.5,61.9,-,"16,099",...,62.1,61.6,-,"16,099",24.2,17%,32 : 68,2016,-0.3,-0.3
3,167,Aarhus University,Denmark,38.1,33.4,55.6,57.3,61.5,49.9,"23,895",...,79.8,68.3,57.7,"23,895",13.6,14%,54 : 46,2016,6.8,6.8
4,276-300,Aberystwyth University,United Kingdom,19.8,63.8,15.5,56.6,35.5,-,"9,252",...,67.2,31.3,-,"9,252",19.2,18%,48 : 52,2016,-4.2,-4.2


In [54]:
merged[['difference_for', 'difference']]

,difference_for,difference
0,data not available,NaN
1,7.3,7.3
2,-0.3,-0.3
3,6.8,6.8
4,-4.2,-4.2
...,...,...
811,6.4,6.4
812,5.6,5.6
813,data not available,NaN
814,27.4,27.4


## 3. Find the university with the largest increase computed in the previous point

In [55]:
merged[merged['difference_for'] != 'data not available'].sort_values('difference_for', ascending = False)

,world_rank_min,university_name,country,teaching_min,international_min,research_min,citations_min,income_min,total_score_min,num_students_min,...,citations_max,income_max,total_score_max,num_students_max,student_staff_ratio_max,international_students_max,female_male_ratio_max,year_max,difference_for,difference
427,251-275,TU Dresden,Germany,27.3,49.2,13.8,57.4,31.9,-,"35,487",...,66.1,99.7,52.1,"35,487",37.4,12%,42 : 58,2016,67.8,67.8
277,174,Nanyang Technological University,Singapore,43.6,96.3,51.7,45.0,40.0,49.0,"25,028",...,85.6,99.9,68.2,"25,028",16.2,33%,48 : 52,2016,59.9,59.9
229,61,LMU Munich,Germany,59.1,43.1,57.5,76.4,40.4,63.0,"35,691",...,85.7,100.0,77.3,"35,691",15.5,13%,62 : 38,2016,59.6,59.6
204,187,Karlsruhe Institute of Technology,Germany,45.0,47.3,35.4,60.7,40.0,47.2,"25,294",...,73.8,99.5,54.5,"25,294",24.6,16%,26 : 74,2016,59.5,59.5
275,201-225,Nagoya University,Japan,45.5,21.2,39.2,43.8,33.1,-,"15,529",...,40.1,91.4,-,"15,529",7.9,10%,29 : 71,2016,58.3,58.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
434,122,Technical University of Denmark,Denmark,46.2,64.0,46.9,64.6,95.5,54.5,"9,990",...,77.8,50.0,51.1,"9,990",5.0,18%,27 : 73,2016,-45.5,-45.5
585,276-300,University of Iceland,Iceland,10.7,56.9,17.3,62.4,75.4,-,"13,960",...,91.4,28.0,-,"13,960",25.9,8%,66 : 34,2016,-47.4,-47.4
236,124,Leiden University,Netherlands,47.3,40.0,54.9,59.3,100.0,54.4,"21,222",...,85.2,49.8,65.7,"21,222",17.1,10%,59 : 41,2016,-50.2,-50.2
490,95,University of Arizona,United States of America,52.4,21.9,52.2,70.1,84.2,57.3,"36,429",...,79.5,32.4,51.7,"36,429",12.7,8%,52 : 48,2016,-51.8,-51.8


In [56]:
#soluzione con ciclo for

temp = -1000

for i in range(1, len(merged)-1):
    
    if merged.iloc[i]['difference'] != 'data not available':
        if merged.iloc[i]['difference'] > temp:
                temp = merged.iloc[i]['difference']
                index = i
        
merged.iloc[index]

world_rank_min                   251-275
university_name               TU Dresden
country                          Germany
teaching_min                        27.3
international_min                   49.2
research_min                        13.8
citations_min                       57.4
income_min                          31.9
total_score_min                        -
num_students_min                  35,487
student_staff_ratio_min             37.4
international_students_min           12%
female_male_ratio_min            42 : 58
year_min                            2012
world_rank_max                      =158
teaching_max                        41.4
international_max                   47.7
research_max                        45.8
citations_max                       66.1
income_max                          99.7
total_score_max                     52.1
num_students_max                  35,487
student_staff_ratio_max             37.4
international_students_max           12%
female_male_rati

In [57]:
merged.iloc[merged['difference'].idxmax()]['university_name']

'TU Dresden'

In [58]:
merged[['income_max', 'income_min', 'difference', 'difference_for']]

,income_max,income_min,difference,difference_for
0,NaN,NaN,NaN,data not available
1,43.7,36.4,7.3,7.3
2,61.6,61.9,-0.3,-0.3
3,68.3,61.5,6.8,6.8
4,31.3,35.5,-4.2,-4.2
...,...,...,...,...
811,37.1,30.7,6.4,6.4
812,31.7,26.1,5.6,5.6
813,82.3,NaN,NaN,data not available
814,65.4,38.0,27.4,27.4


## 4. For each ranking, consider only the most recent data point. For each university, compute the maximum difference between the rankings (e.g. for Aarhus University the value is 122-73=49). Notice that some rankings are expressed as a range

In [70]:
shanghai_max_year = shanghai.iloc[shanghai.groupby('university_name').idxmax()['year']][['world_rank', 'university_name']]
shanghai_max_year.set_index('university_name', inplace = True)
shanghai_max_year.head()

,world_rank
university_name,
Aalborg University,301-400
Aalto University,401-500
Aarhus University,73
Aix Marseille University,101-150
Aix-Marseille University,102-150


In [69]:
cwur_max_year = cwur.iloc[cwur.groupby('institution').idxmax()['year']][['world_rank', 'institution']]
cwur_max_year.set_index('institution', inplace = True)
cwur_max_year.head()

,world_rank
institution,
AGH University of Science and Technology,782
Aalborg University,565
Aalto University,421
Aarhus University,122
Aberystwyth University,814


In [71]:
times_max_year.set_index('university_name', inplace = True)

In [65]:
from functools import reduce

In [78]:
datasets = [times_max_year['world_rank'], shanghai_max_year, cwur_max_year]

In [79]:
rankings = reduce(lambda left, right: pd.merge(left, right, left_index = True, right_index = True, how = 'outer'), datasets)
rankings.head()

,world_rank_x,world_rank_y,world_rank
AGH University of Science and Technology,601-800,NaN,782.0
Aalborg University,201-250,301-400,565.0
Aalto University,251-300,401-500,421.0
Aarhus University,=106,73,122.0
Aberystwyth University,301-350,NaN,814.0


In [84]:
def newrank(string):
    #if string.isnull(): 
    if type(string) != str:
        return string
    else:
        occ1 = re.search('(?P<min>\d+)-(?P<max>\d+)', string)
        occ2 = re.search('^\D?(?P<num>\d+)$', string)
        if occ1:
            #per i range consideriamo la media?
            return (float(occ1.group('min')) + float(occ1.group('max')))/2
        elif occ2:
            return float(occ2.group('num')) 

In [87]:
rankings['world_rank_x_num'] = rankings['world_rank_x'].apply(newrank)
rankings['world_rank_y_num'] = rankings['world_rank_y'].apply(newrank)
rankings 
#pop?

,world_rank_x,world_rank_y,world_rank,world_rank_x_num,world_rank_y_num
AGH University of Science and Technology,601-800,NaN,782.0,700.5,NaN
Aalborg University,201-250,301-400,565.0,225.5,350.5
Aalto University,251-300,401-500,421.0,275.5,450.5
Aarhus University,=106,73,122.0,106.0,73.0
Aberystwyth University,301-350,NaN,814.0,325.5,NaN
...,...,...,...,...,...
École centrale de Lyon,NaN,NaN,881.0,NaN,NaN
École normale supérieure - Paris,NaN,NaN,37.0,NaN,NaN
École normale supérieure de Cachan,NaN,NaN,721.0,NaN,NaN
École normale supérieure de Lyon,NaN,NaN,471.0,NaN,NaN


In [88]:
rankings['world_rank'].dtype

dtype('float64')

In [91]:
def absolute_difference(row):
    item1 = row['world_rank_x_num']
    item2 = row['world_rank_y_num']
    item3 = row['world_rank']
    return max(abs(item1 - item2), abs(item1 - item3), abs(item3 - item2))

In [94]:
rankings['absolute_difference'] = rankings.apply(absolute_difference, axis = 1)
rankings

,world_rank_x,world_rank_y,world_rank,world_rank_x_num,world_rank_y_num,difference,absolute_difference
AGH University of Science and Technology,601-800,NaN,782.0,700.5,NaN,NaN,NaN
Aalborg University,201-250,301-400,565.0,225.5,350.5,339.5,339.5
Aalto University,251-300,401-500,421.0,275.5,450.5,175.0,175.0
Aarhus University,=106,73,122.0,106.0,73.0,49.0,49.0
Aberystwyth University,301-350,NaN,814.0,325.5,NaN,NaN,NaN
...,...,...,...,...,...,...,...
École centrale de Lyon,NaN,NaN,881.0,NaN,NaN,NaN,NaN
École normale supérieure - Paris,NaN,NaN,37.0,NaN,NaN,NaN,NaN
École normale supérieure de Cachan,NaN,NaN,721.0,NaN,NaN,NaN,NaN
École normale supérieure de Lyon,NaN,NaN,471.0,NaN,NaN,NaN,NaN


## 5. Consider only the most recent data point of the times dataset. Compute the number of male and female students for each country.

## 6. Find the universities where the ratio between female and male is below the average ratio (computed over all universities)

## 7. For each country, compute the fraction of the students in the country that are in one of the universities computed in the previous point (that is, the denominator of the ratio is the total number of students over all universities in the country).

## 8. Read the file educational_attainment_supplementary_data.csv, discarding any row with missing country_name or series_name

## 9. From attainment build a dataframe with the same data, but with 4 columns: country_name, series_name, year, value

## 10. For each university, find the number of rankings in which they appear (it suffices to appear in one year for each ranking).

## 11. In the times ranking, compute the number of times each university appears

## 12. Find the universities that appear at most twice in the times ranking.

## 13. The universities that, in any year, have the same position in all three rankings (they must have the same position in a year).